Note: this notebook is set up to run with the env.yml containing the name 'polaris_datasets'

# Dataset creation with [Polaris](https://github.com/polaris-hub/polaris) 

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import pathlib

import pandas as pd
import datamol as dm

# polaris dataset
from polaris.dataset import Dataset, ColumnAnnotation

from polaris.utils.types import HubOwner


root = pathlib.Path("__file__").absolute().parents[2]
os.chdir(root)
sys.path.insert(0, str(root))
from utils.docs_utils import load_readme

In [15]:
# Get the owner and organization
org = "Graphium"
data_name = "tox21"
dirname = dm.fs.join(root, f"org-{org}", data_name)
gcp_root = f"gs://polaris-public/polaris-recipes/org-{org}/{data_name}"

owner = HubOwner(slug=org.lower(), type="organization")
owner

HubOwner(slug='graphium', external_id=None, type='organization')

In [16]:
BENCHMARK_DIR = f"{gcp_root}/benchmarks"
DATASET_DIR = f"{gcp_root}/datasets"
FIGURE_DIR = f"{gcp_root}/figures"

## Load existing data

In [3]:
PATH = f"{gcp_root}/data/raw/Tox21-7k-12-labels.csv"
table = pd.read_csv(PATH)
table.columns

NameError: name 'gcp_root' is not defined

### Below we specify the meta information of data columns

In [10]:
# Additional meta-data on the column level
annotations = {
    "smiles": ColumnAnnotation(
        description="Molecule SMILES string", modality="molecule"
    ),
    "NR-AR": ColumnAnnotation(description="NR-AR"),
    "NR-AR-LBD": ColumnAnnotation(description="NR-AR-LBD"),
    "NR-AhR": ColumnAnnotation(description="NR-AhR"),
    "NR-Aromatase": ColumnAnnotation(description="NR-Aromatase"),
    "NR-ER": ColumnAnnotation(description="NR-ER"),
    "NR-ER-LBD": ColumnAnnotation(description="NR-ER-LBD"),
    "NR-PPAR-gamma": ColumnAnnotation(description="NR-PPAR-gamma"),
    "SR-ARE": ColumnAnnotation(description="SR-ARE"),
    "SR-ATAD5": ColumnAnnotation(description="SR-ATAD5"),
    "SR-HSE": ColumnAnnotation(description="SR-HSE"),
    "SR-MMP": ColumnAnnotation(description="SR-MMP"),
    "SR-p53": ColumnAnnotation(description="SR-p53"),
}

### Define `Dataset` object

In [17]:
version = "v1"
dataset_name = f"{data_name}-{version}"

In [18]:
dataset = Dataset(
    # The table is the core data-structure required to construct a dataset
    table=table[annotations.keys()],
    # Additional meta-data on the dataset level.
    name=dataset_name,
    description="Tox21 challenge, a dataset with 12,707 chemical compounds was given. This dataset consisted of a training dataset of 11,764, a leaderboard set of 296, and a test set of 647 compounds.",
    source="https://www.frontiersin.org/research-topics/2954/tox21-challenge-to-build-predictive-models-of-nuclear-receptor-and-stress-response-pathways-as-mediated-by-exposure-to-environmental-toxicants-and-drugs/overview",
    annotations=annotations,
    tags=["Toxicity"],
    owner=owner,
    license="CC-BY-4.0",
    user_attributes={"year": "2017"},
    # readme=load_readme("org-Polaris/astra_zeneca/logD_74/AZ_logD_readme.md"),
    # curation_reference="https://github.com/polaris-hub/polaris-recipes/org-Polaris/astra_zeneca/logD_74/01_AZ_logD_data_curation.ipynb",
)

In [19]:
# save the dataset to GCP
SAVE_DIR = f"{DATASET_DIR}/{dataset_name}"
dataset_path = dataset.to_json(SAVE_DIR)
dataset_path

2024-07-16 21:56:21.884 | INFO     | polaris._mixins:md5sum:27 - Computing the checksum. This can be slow for large datasets.


'gs://polaris-public/polaris-recipes/org-Graphium/tox21/datasets/tox21-v1/dataset.json'

In [20]:
from polaris.hub.client import PolarisHubClient

client = PolarisHubClient()
client.login()

client.upload_dataset(dataset=dataset, access="public", owner=owner)

2024-07-16 21:56:30.728 | INFO     | polaris.hub.client:login:285 - You are already logged in to the Polaris Hub as  (lu@valencediscovery.com). Set `overwrite=True` to force re-authentication.
2024-07-16 21:56:32.771 | SUCCESS  | polaris.hub.client:upload_dataset:631 - Your dataset has been successfully uploaded to the Hub. View it here: https://polarishub.io/datasets/graphium/tox21-v1


{'id': '60AuT0meGDMM9i4p8TVUw',
 'createdAt': '2024-07-17T01:56:31.353Z',
 'deletedAt': None,
 'name': 'tox21-v1',
 'slug': 'tox21-v1',
 'description': 'Tox21 challenge, a dataset with 12,707 chemical compounds was given. This dataset consisted of a training dataset of 11,764, a leaderboard set of 296, and a test set of 647 compounds.',
 'tags': ['Toxicity'],
 'userAttributes': {'year': '2017'},
 'access': 'public',
 'isCertified': False,
 'polarisVersion': 'dev',
 'readme': '',
 'ownerId': 'zMTB7lQiiukqEmLQF7EjT',
 'creatorId': 'NKnaHGybLqwSHcaMEHqfF',
 'state': 'upload_pending',
 'source': 'https://www.frontiersin.org/research-topics/2954/tox21-challenge-to-build-predictive-models-of-nuclear-receptor-and-stress-response-pathways-as-mediated-by-exposure-to-environmental-toxicants-and-drugs/overview',
 'curationReference': None,
 'nRows': 7831,
 'nColumns': 13,
 'license': 'CC-BY-4.0',
 'md5Sum': '4eacdc90d3361063a7074e48a26f3033',
 'annotations': {'NR-AR': {'dtype': 'float64',
   'mod